In [1]:
#Loading
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
#Modeling
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#Visual
import matplotlib.pyplot as plt
import seaborn as sns
#For TfIdf
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from cycler import cycler

In [2]:
data = pd.read_csv('test_data.csv')
target = data['final_payment_status']
# data = data.drop(columns = 'final_payment_status')

# cols = ['delivery_week']  #'delivery_week', 'payment_method','custom_meal_selection', 'state', 'channel'
# data = pd.get_dummies(data = data, columns = cols)

data = data.drop(columns='avg_recipe_rating')
data = data.drop(columns='time_to_pay')
# data = data.dropna(subset=['time_to_pay', 'customer_id'])
# data['final_payment_status']


In [3]:
target.unique()

array(['Cancelled', 'Paid', 'Unpaid', 'Other', 'Refunded'], dtype=object)

In [4]:
data.dtypes

customer_id                 int64
order_number                int64
delivery_date               int64
delivery_week              object
final_payment_status       object
payment_method             object
custom_meal_selection      object
state                      object
channel                    object
engagement_score            int64
num_prior_orders_failed     int64
num_prior_orders_unpaid     int64
dtype: object

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   customer_id              4998 non-null   int64 
 1   order_number             4998 non-null   int64 
 2   delivery_date            4998 non-null   int64 
 3   delivery_week            4998 non-null   object
 4   final_payment_status     4998 non-null   object
 5   payment_method           4998 non-null   object
 6   custom_meal_selection    4998 non-null   object
 7   state                    4998 non-null   object
 8   channel                  4998 non-null   object
 9   engagement_score         4998 non-null   int64 
 10  num_prior_orders_failed  4998 non-null   int64 
 11  num_prior_orders_unpaid  4998 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 468.7+ KB


In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
model_OHE = ColumnTransformer([('finalPay', OneHotEncoder(),[4])], remainder = 'passthrough')
model_OHE2 = ColumnTransformer([('PayMethod', OneHotEncoder(),['payment_method'])], remainder = 'passthrough')
model_OHE3 = ColumnTransformer([('State', OneHotEncoder(),['state'])], remainder = 'passthrough')
model_OHE4 = ColumnTransformer([('Channel', OneHotEncoder(),['channel'])], remainder = 'passthrough')
model_OHE5 = ColumnTransformer([('DelWeek', OneHotEncoder(),['delivery_week'])], remainder = 'passthrough')
model_OHE6 = ColumnTransformer([('CustomMeal', OneHotEncoder(),['custom_meal_selection'])], remainder = 'passthrough')
data['final_payment_status'] = model_OHE.fit_transform(data)
data['payment_method'] = model_OHE2.fit_transform(data)
data['state'] = model_OHE3.fit_transform(data)
data['channel'] = model_OHE4.fit_transform(data)
data['delivery_week'] = model_OHE5.fit_transform(data)
data['custom_meal_selection'] = model_OHE6.fit_transform(data)

In [7]:
data['final_payment_status'] = data['final_payment_status'].astype(float)
data['payment_method'] = data['payment_method'].astype(float)
data['state'] = data['state'].astype(float)
data['channel'] = data['channel'].astype(float)
data['delivery_week'] = data['delivery_week'].astype(float)
data['custom_meal_selection'] = data['custom_meal_selection'].astype(float)

In [8]:
data.dtypes

customer_id                  int64
order_number                 int64
delivery_date                int64
delivery_week              float64
final_payment_status       float64
payment_method             float64
custom_meal_selection      float64
state                      float64
channel                    float64
engagement_score             int64
num_prior_orders_failed      int64
num_prior_orders_unpaid      int64
dtype: object

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              4998 non-null   int64  
 1   order_number             4998 non-null   int64  
 2   delivery_date            4998 non-null   int64  
 3   delivery_week            4998 non-null   float64
 4   final_payment_status     4998 non-null   float64
 5   payment_method           4998 non-null   float64
 6   custom_meal_selection    4998 non-null   float64
 7   state                    4998 non-null   float64
 8   channel                  4998 non-null   float64
 9   engagement_score         4998 non-null   int64  
 10  num_prior_orders_failed  4998 non-null   int64  
 11  num_prior_orders_unpaid  4998 non-null   int64  
dtypes: float64(6), int64(6)
memory usage: 468.7 KB


In [10]:
data

,customer_id,order_number,delivery_date,delivery_week,final_payment_status,payment_method,custom_meal_selection,state,channel,engagement_score,num_prior_orders_failed,num_prior_orders_unpaid
0,6,17266,20200229,1.0,1.0,0.0,0.0,0.0,0.0,3,1,0
1,7,10494,20200301,1.0,0.0,0.0,0.0,0.0,0.0,29,14,0
2,8,10384,20200303,1.0,0.0,0.0,0.0,0.0,0.0,17,9,0
3,9,3396,20200303,1.0,0.0,0.0,0.0,0.0,0.0,242,3,0
4,13,1146,20200302,1.0,0.0,0.0,0.0,0.0,0.0,226,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4993,17922,21627,20200303,1.0,0.0,0.0,1.0,0.0,0.0,13,4,0
4994,17924,11742,20200303,1.0,0.0,0.0,0.0,0.0,0.0,3,0,0
4995,17926,17142,20200301,1.0,0.0,0.0,0.0,0.0,0.0,3,2,0
4996,17932,1157,20200302,1.0,0.0,0.0,0.0,0.0,0.0,11,0,0


In [11]:
target = data['final_payment_status']

In [12]:
target.unique()

array([1., 0.])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,
                                                          random_state=15)

In [14]:
X_train

,customer_id,order_number,delivery_date,delivery_week,final_payment_status,payment_method,custom_meal_selection,state,channel,engagement_score,num_prior_orders_failed,num_prior_orders_unpaid
160,633,12154,20200229,1.0,0.0,0.0,0.0,0.0,0.0,5,6,0
1610,5744,6941,20200229,1.0,0.0,0.0,0.0,0.0,0.0,9,5,0
2038,7251,1149,20200304,1.0,0.0,0.0,0.0,0.0,0.0,50,15,0
3398,12059,16420,20200302,1.0,0.0,0.0,0.0,0.0,0.0,45,25,0
4807,17306,10180,20200303,1.0,0.0,0.0,0.0,0.0,0.0,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3271,11606,13614,20200303,1.0,0.0,0.0,1.0,0.0,0.0,10,3,0
2715,9634,19352,20200229,1.0,0.0,0.0,0.0,0.0,0.0,55,0,0
2204,7782,19632,20200229,1.0,0.0,0.0,0.0,0.0,0.0,3,2,2
2693,9526,14440,20200229,1.0,0.0,0.0,0.0,0.0,0.0,51,8,4


In [15]:
X_test

,customer_id,order_number,delivery_date,delivery_week,final_payment_status,payment_method,custom_meal_selection,state,channel,engagement_score,num_prior_orders_failed,num_prior_orders_unpaid
2125,7516,1472,20200302,1.0,0.0,0.0,0.0,0.0,0.0,37,39,0
2076,7357,11991,20200304,1.0,0.0,0.0,0.0,0.0,0.0,35,24,0
4443,15997,18743,20200229,1.0,0.0,0.0,0.0,0.0,0.0,5,3,0
2949,10529,13559,20200229,1.0,0.0,0.0,0.0,0.0,0.0,4,0,0
847,3060,21693,20200229,1.0,0.0,0.0,1.0,0.0,0.0,29,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2358,8342,8471,20200302,1.0,0.0,0.0,1.0,0.0,0.0,74,36,0
520,1915,12451,20200229,1.0,0.0,0.0,0.0,0.0,0.0,8,7,0
634,2298,11308,20200229,1.0,0.0,0.0,0.0,0.0,0.0,36,2,0
3495,12441,12830,20200303,1.0,0.0,0.0,0.0,0.0,0.0,8,8,1


In [16]:
y_train

160     0.0
1610    0.0
2038    0.0
3398    0.0
4807    0.0
       ... 
3271    0.0
2715    0.0
2204    0.0
2693    0.0
3829    0.0
Name: final_payment_status, Length: 3998, dtype: float64

In [17]:
y_test

2125    0.0
2076    0.0
4443    0.0
2949    0.0
847     0.0
       ... 
2358    0.0
520     0.0
634     0.0
3495    0.0
3079    0.0
Name: final_payment_status, Length: 1000, dtype: float64

In [18]:
# data['final_payment_status']

In [19]:
from sklearn.ensemble import RandomForestRegressor
# Модель леса из 10 деревьев
rndforest = RandomForestRegressor(n_estimators=10, oob_score=True, random_state=15)
rndforest.fit(X_train, y_train)

c:\Users\tuska\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestRegressor(n_estimators=10, oob_score=True, random_state=15)

In [20]:
# Действующая классификация
pred_rndforest = rndforest.predict(X_test)

In [21]:
print(classification_report(y_test, pred_rndforest, zero_division='warn'))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       970
         1.0       1.00      1.00      1.00        30

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

